# Day4 lunch exercise

In [2]:
import pandas as pd

### Load and parse data

In [3]:
df_gtf = pd.read_csv('BDGP6.Ensembl.81.gtf', sep = '\t', header = None, skiprows = 5)
df_gtf.columns = ['Chr', 'Src', 'Feature', 'Start', 'End', 'Score', 'Strand', 'Frame', 'Attributes']
df_gtf

,Chr,Src,Feature,Start,End,Score,Strand,Frame,Attributes
0,3R,FlyBase,gene,722370,722621,.,-,.,"gene_id ""FBgn0085804""; gene_version ""1""; gene_..."
1,3R,FlyBase,transcript,722370,722621,.,-,.,"gene_id ""FBgn0085804""; gene_version ""1""; trans..."
2,3R,FlyBase,exon,722370,722621,.,-,.,"gene_id ""FBgn0085804""; gene_version ""1""; trans..."
3,3R,FlyBase,gene,835381,2503907,.,+,.,"gene_id ""FBgn0267431""; gene_version ""1""; gene_..."
4,3R,FlyBase,transcript,835381,2503907,.,+,.,"gene_id ""FBgn0267431""; gene_version ""1""; trans..."
...,...,...,...,...,...,...,...,...,...
538334,211000022279342,FlyBase,transcript,1,530,.,+,.,"gene_id ""FBgn0085772""; gene_version ""1""; trans..."
538335,211000022279342,FlyBase,exon,1,530,.,+,.,"gene_id ""FBgn0085772""; gene_version ""1""; trans..."
538336,211000022279132,FlyBase,gene,1,1005,.,+,.,"gene_id ""FBgn0085825""; gene_version ""1""; gene_..."
538337,211000022279132,FlyBase,transcript,1,1005,.,+,.,"gene_id ""FBgn0085825""; gene_version ""1""; trans..."


## Q1. Search for the nearest protein coding gene by binary search

In [4]:
select = (df_gtf.loc[:, 'Chr'] == '3R') & (df_gtf.loc[:, 'Attributes'].str.contains('gene_biotype \"protein_coding\"'))
df_gtf_coding = df_gtf.loc[select]
df_gtf_coding

,Chr,Src,Feature,Start,End,Score,Strand,Frame,Attributes
3,3R,FlyBase,gene,835381,2503907,.,+,.,"gene_id ""FBgn0267431""; gene_version ""1""; gene_..."
4,3R,FlyBase,transcript,835381,2503907,.,+,.,"gene_id ""FBgn0267431""; gene_version ""1""; trans..."
5,3R,FlyBase,exon,835381,835491,.,+,.,"gene_id ""FBgn0267431""; gene_version ""1""; trans..."
6,3R,FlyBase,CDS,835381,835491,.,+,0,"gene_id ""FBgn0267431""; gene_version ""1""; trans..."
7,3R,FlyBase,start_codon,835381,835383,.,+,0,"gene_id ""FBgn0267431""; gene_version ""1""; trans..."
...,...,...,...,...,...,...,...,...,...
124783,3R,FlyBase,exon,32057016,32059145,.,-,.,"gene_id ""FBgn0002645""; gene_version ""1""; trans..."
124784,3R,FlyBase,CDS,32057325,32059145,.,-,0,"gene_id ""FBgn0002645""; gene_version ""1""; trans..."
124785,3R,FlyBase,stop_codon,32057322,32057324,.,-,0,"gene_id ""FBgn0002645""; gene_version ""1""; trans..."
124786,3R,FlyBase,UTR,32067997,32068441,.,-,.,"gene_id ""FBgn0002645""; gene_version ""1""; trans..."


In [5]:
def get_genename(nrow, df):
    '''Retrieve gene name from a gtf-generated data frame.'''
    gname = df.iloc[nrow, 8].rstrip().split(sep = 'gene_name \"')[1].rstrip().split('\"')[0]
    return gname

In [6]:
pos = 21378950

### Binary search

In [32]:
mid = df_gtf_coding.shape[0] // 2
upper = df_gtf_coding.shape[0] - 1
lower = 0
start = df_gtf_coding.iloc[mid, 3]
end = df_gtf_coding.iloc[mid, 4]


In [33]:
time = 0
while (upper - lower) > 1:
    if start > pos:
        upper = mid
        mid = (upper + lower) // 2
    elif end < pos:
        lower = mid
        mid = (upper + lower) // 2
    else:
        upper = lower
    start = df_gtf_coding.iloc[mid, 3]
    end = df_gtf_coding.iloc[mid, 4]
    time += 1
    print(df_gtf_coding.iloc[upper, 3], 
          df_gtf_coding.iloc[mid, 3], 
          df_gtf_coding.iloc[lower, 3])        #Keep tracing the search progress.
print(get_genename(upper, df_gtf_coding), 
         get_genename(mid, df_gtf_coding), 
         get_genename(lower, df_gtf_coding))
print(time)

32057016 24668146 18180685
24668146 21740105 18180685
21740105 20048627 18180685
21740105 20865916 20048627
21740105 21178278 20865916
21740105 21355692 21178278
21740105 21524188 21355692
21524188 21381828 21355692
21381828 21366974 21355692
21381828 21367910 21366974
21381828 21372142 21367910
21381828 21373079 21372142
21381828 21379247 21373079
21379247 21370918 21373079
21379247 21378977 21370918
21378977 21378977 21370918
21378977 21370918 21370918
tin pre-mod(mdg4)-N pre-mod(mdg4)-N
17


In [34]:
def dist(nrow, df, pos):
    '''When overlapping, the distance is set to 0. In other cases, the distance is the minimum of
    the distances from start position to query site, and end point to query site.'''
    if (df.iloc[nrow, 3] - pos) * (df.iloc[nrow, 4] - pos) <= 0:
        return 0
    dist = min(abs(df.iloc[nrow, 3] - pos), abs(df.iloc[nrow, 4] - pos))
    return dist

In [35]:
distance_upper = dist(upper, df_gtf_coding, pos)
distance_lower = dist(lower, df_gtf_coding, pos)

In [36]:
if distance_upper > distance_lower:
    print(get_genename(lower, df_gtf_coding), distance_lower)
else:
    print(get_genename(upper, df_gtf_coding), distance_upper)

tin 27


### Therefore the protein coding gene nearest to 3R: 21378950 is *tin* and the linear distance is 27 bp.

## Adv. Q2. Find 20 nearest genes to 3R: 21378950

In [37]:
nearest_genes_coding=[]
line_up, line_low = upper, upper
nearest_genes_coding.append(get_genename(line_up, df_gtf_coding))
ngenes = len(nearest_genes_coding)
distance = dist(line_up, df_gtf_coding, pos)

while ngenes < 21:
    line_up += 1
    line_low -= 1
    if get_genename(line_up, df_gtf_coding) not in nearest_genes_coding:
        nearest_genes_coding.append(get_genename(line_up, df_gtf_coding))
        ngenes += 1
        if dist(line_up, df_gtf_coding, pos) < distance:   #Make sure that the last gene in nearest_genes_coding is always the furthest to the site of interests.
            nearest_genes_coding[-1], nearest_genes_coding[-2] = nearest_genes_coding[-2], nearest_genes_coding[-1]
        else:
            distance = dist(line_up, df_gtf_coding, pos)
    if get_genename(line_low, df_gtf_coding) not in nearest_genes_coding:
        nearest_genes_coding.append(get_genename(line_low, df_gtf_coding))
        ngenes += 1
        if dist(line_low, df_gtf_coding, pos) < distance:
            nearest_genes_coding[-1], nearest_genes_coding[-2] = nearest_genes_coding[-2], nearest_genes_coding[-1]
        else:
            distance = dist(line_low, df_gtf_coding, pos)

print(nearest_genes_coding)  #In the order that the linear distance to the position of interest gradually increases.

['tin', 'pre-mod(mdg4)-N', 'pre-mod(mdg4)-AA', 'bap', 'pre-mod(mdg4)-O', 'pre-mod(mdg4)-V', 'CG6475', 'pre-mod(mdg4)-AD', 'pre-mod(mdg4)-X', 'CG7907', 'CheB93a', 'pre-mod(mdg4)-W', 'CheB93b', 'pre-mod(mdg4)-Y', 'pre-mod(mdg4)-U', 'pre-mod(mdg4)-AE', 'lbl', 'lbe', 'pre-mod(mdg4)-AB', 'pre-mod(mdg4)-C', 'CG7922']


## Adv. Q3. Find the nearest gene without the limitation in gene type

In [38]:
select = (df_gtf.loc[:, 'Chr'] == '3R')
df_gtf_3R = df_gtf[select]
df_gtf_3R

,Chr,Src,Feature,Start,End,Score,Strand,Frame,Attributes
0,3R,FlyBase,gene,722370,722621,.,-,.,"gene_id ""FBgn0085804""; gene_version ""1""; gene_..."
1,3R,FlyBase,transcript,722370,722621,.,-,.,"gene_id ""FBgn0085804""; gene_version ""1""; trans..."
2,3R,FlyBase,exon,722370,722621,.,-,.,"gene_id ""FBgn0085804""; gene_version ""1""; trans..."
3,3R,FlyBase,gene,835381,2503907,.,+,.,"gene_id ""FBgn0267431""; gene_version ""1""; gene_..."
4,3R,FlyBase,transcript,835381,2503907,.,+,.,"gene_id ""FBgn0267431""; gene_version ""1""; trans..."
...,...,...,...,...,...,...,...,...,...
124783,3R,FlyBase,exon,32057016,32059145,.,-,.,"gene_id ""FBgn0002645""; gene_version ""1""; trans..."
124784,3R,FlyBase,CDS,32057325,32059145,.,-,0,"gene_id ""FBgn0002645""; gene_version ""1""; trans..."
124785,3R,FlyBase,stop_codon,32057322,32057324,.,-,0,"gene_id ""FBgn0002645""; gene_version ""1""; trans..."
124786,3R,FlyBase,UTR,32067997,32068441,.,-,.,"gene_id ""FBgn0002645""; gene_version ""1""; trans..."


### Binary search based on start position

In [39]:
mid = df_gtf_3R.shape[0] // 2
upper = df_gtf_3R.shape[0] - 1
lower = 0
start = df_gtf_3R.iloc[mid, 3]
end = df_gtf_3R.iloc[mid, 4]

In [40]:
time = 0
while (upper - lower) > 1:
    if start > pos:
        upper = mid
        mid = (upper + lower) // 2
    elif end < pos:
        lower = mid
        mid = (upper + lower) // 2
    else:
        upper = lower
    start = df_gtf_3R.iloc[mid, 3]
    end = df_gtf_3R.iloc[mid, 4]
    time += 1
    print(df_gtf_3R.iloc[upper, 3],
          df_gtf_3R.iloc[mid, 3],
          df_gtf_3R.iloc[lower, 3])     #Keep tracing search progress.
print(get_genename(upper, df_gtf_3R), 
         get_genename(mid, df_gtf_3R), 
         get_genename(lower, df_gtf_3R))
print(time)

32057016 24705797 18177094
24705797 21666253 18177094
21666253 20042935 18177094
21666253 20844627 20042935
21666253 21154927 20844627
21666253 21354445 21154927
21666253 21499986 21354445
21499986 21366974 21354445
21499986 21404098 21366974
21404098 21373079 21366974
21404098 21399723 21373079
21399723 21381828 21373079
21381828 21379247 21373079
21379247 21370918 21373079
21379247 21378977 21370918
21378977 21378977 21370918
21378977 21370918 21370918
tin pre-mod(mdg4)-N pre-mod(mdg4)-N
17


### The results are the same no matter whether there is a limitation in gene type.

## Adv. Q4. Find nearest genes at sites in a given list

In [41]:
df_chr = dict()
for chromosome in df_gtf.loc[:, 'Chr'].unique():
    df_chr[chromosome] = df_gtf.loc[df_gtf.loc[:, 'Chr'] == chromosome]
df_chr

{'3R':        Chr      Src     Feature     Start       End Score Strand Frame  \
 0       3R  FlyBase        gene    722370    722621     .      -     .   
 1       3R  FlyBase  transcript    722370    722621     .      -     .   
 2       3R  FlyBase        exon    722370    722621     .      -     .   
 3       3R  FlyBase        gene    835381   2503907     .      +     .   
 4       3R  FlyBase  transcript    835381   2503907     .      +     .   
 ...     ..      ...         ...       ...       ...   ...    ...   ...   
 124783  3R  FlyBase        exon  32057016  32059145     .      -     .   
 124784  3R  FlyBase         CDS  32057325  32059145     .      -     0   
 124785  3R  FlyBase  stop_codon  32057322  32057324     .      -     0   
 124786  3R  FlyBase         UTR  32067997  32068441     .      -     .   
 124787  3R  FlyBase         UTR  32057016  32057321     .      -     .   
 
                                                Attributes  
 0       gene_id "FBgn0085804";

In [51]:
positions = open('day3_100_positions.txt', 'r')
positions.seek(0, 0)

0

In [52]:
for line in positions:
    chromosome, pos = line.rstrip().split()
    pos = int(pos)
    df = df_chr[chromosome]

    mid = df.shape[0] // 2
    upper = df.shape[0] - 1
    lower = 0
    start = df.iloc[mid, 3]
    end = df.iloc[mid, 4]
    time = 0
    while (upper - lower) > 1:
        if start > pos:
            upper = mid
            mid = (upper + lower) // 2
        elif end < pos:
            lower = mid
            mid = (upper + lower) // 2
        else:
            upper = lower
        start = df.iloc[mid, 3]
        end = df.iloc[mid, 4]
        time += 1
    if dist(upper, df, pos) < dist(lower, df, pos):
        print(get_genename(upper, df))
    elif dist(upper, df, pos) > dist(lower, df, pos):
        print(get_genename(lower, df))
    else:
        print(get_genename(upper, df), get_genename(lower, df))
    print(time, '\n')

CG31835
17 

CR44739
17 

CG43331
17 

CG17193
17 

Snup
17 

tRNA:CR32173
17 

Ntf-2 Ntf-2
7 

RpL14
17 

Tsc1
17 

CG33658
17 

CR45279
17 

CG10483
17 

feo
17 

CG8321 CG8321
14 

CG33965
17 

GalNAc-T1
17 

CG4169
17 

CG31496
17 

CG31661
17 

RpL14
17 

tRNA:S7:64D
17 

Catsup
17 

CR45656
17 

Kap-alpha1
16 

eIF4G
13 

TfAP-2
17 

CG30177 CG30177
15 

CG7686
16 

CG4164
17 

CR42746
17 

CG3769 CG3769
17 

Cyp4d21
17 

CR43621
16 

Cha
17 

CG8031
17 

CR45405
17 

bab2
17 

RhoGAPp190
16 

CG8974
16 

Npc2g
16 

eIF4E-3
16 

5SrRNA:CR33442
17 

Hmt-1 Hmt-1
16 

hog
17 

CG11664
17 

uri
17 

otk
17 

Ubp64E Ubp64E
10 

Irp-1B
17 

CG8281
17 

CG9541
17 

CG6845
17 

CR44339
17 

CG1532
17 

mRpL10
17 

CG6543 CG6543
15 

CG30495
17 

CG15153
16 

CG33331
17 

CG31546 CG31546
13 

CG2217
17 

CG11334
17 

Mur2B
16 

CG10405
17 

axo
17 

CG31949
16 

CG34113
17 

CR45672
17 

CG9555
16 

CR44457
17 

Hr96
17 

ATPsyn-Cf6
17 

CG16798
17 

CG5569
16 

CR44112
17 

Inos
17 

Ede

In [53]:
positions.close()